In [1]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#ctd
M1=np.zeros((19,106)) #国产
M3=np.zeros((19,106)) #进口
#dts
M2=np.zeros((1,18)) #国产
M4=np.zeros((1,18)) #进口

#仓库状态矩阵
O=np.zeros((20,1))  #是否开仓
Z=np.zeros((20,1))  #是否智能


In [2]:
#参数导入
#国产，对应1
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))
Q_dm.shape

(1, 106)

In [3]:
#港口，对应0
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))


In [4]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,6,axis=0)#dm(delete gangkou)
k = 10 
row_to_move = L_dts_1[k]
L_dts_1 = np.delete(L_dts_1, k, axis=0)
L_dts_1 = np.vstack([row_to_move, L_dts_1])

L_dts_2=np.delete(L_dts,11,axis=0)#im(delete neilu)
k = 6 
row_to_move = L_dts_2[k]
L_dts_2 = np.delete(L_dts_2, k, axis=0)
L_dts_2 = np.vstack([row_to_move, L_dts_2])
L_dts_2

array([[2632.0, 1952.0, 2643.0, ..., 1348.0, 718.0, 1777.0],
       [671.0, 159.0, 662.0, ..., 786.0, 2262.0, 418.0],
       [2116.0, 1343.0, 1861.0, ..., 724.0, 812.0, 1290.0],
       ...,
       [1785.0, 1023.0, 1674.0, ..., 401.0, 1114.0, 957.0],
       [1735.0, 942.0, 835.0, ..., 671.0, 1817.0, 1022.0],
       [1332.0, 539.0, 1081.0, ..., 203.0, 1602.0, 551.0]], dtype=object)

In [5]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,6,axis=0)#dm(delete gangkou)
T_dts_2=np.delete(T_dts,11,axis=0)#im(delete neilu)
T_dts_1.shape

(19, 106)

In [6]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]

array([57.0, 1461.0, 1841.0, 1752.0, 837.0, 2131.0, 2122.0, 1346.0,
       1276.0, 999.0, 1327.0, 651.0, 1154.0, 2147.0, 91.0, 1132.0, 1071.0,
       698.0], dtype=object)

In [7]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]

array([[1332.0, 649.0, 1349.0, 1209.0, 1732.0, 421.0, 449.0, 2024.0,
        696.0, 852.0, 624.0, 1148.0, 801.0, 404.0, 1298.0, 757.0, 1212.0,
        1032.0],
       [110.0, 1026.0, 1279.0, 1247.0, 682.0, 1386.0, 1391.0, 1003.0,
        949.0, 738.0, 976.0, 491.0, 885.0, 1398.0, 115.0, 832.0, 813.0,
        496.0]], dtype=object)

In [8]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1:]
Pf.shape

(20, 2)

In [9]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
Of.shape

(20,)

In [10]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
Cap.shape

(20,)

In [11]:
#约束条件
##时间
Q_all=Q_im.sum()
def time_point(M_im,M_dm,Q_im,Q_dm):
    Q_out=0
    tmp1=M_im*T_dts
    tmp2=M_dm*T_dts
    print(tmp1,"\n",tmp2)
    for i in (0,106):
        if tmp1.sum(axis=1)>=10:
            Q_out+=Q_im[i]
        if tmp2.sum(axis=1)>=10:
            Q_out+=Q_dm[i]
    out_point=Q_out/Q_all
    return out_point

def time_lim(M_im,M_dm,Q_im,Q_dm):
    if time_point(M_im,M_dm,Q_im,Q_dm)<=0.95:
        return True
    return False

#最大订单量不超过最大承载量
def cap_lim(M_im,M_dm):
    tmp=M_im.T*Q_im+M_dm.T*Q_dm
    tmp1=tmp.sum(axis=1)
    for i,j in tmp1,Cap:
        if i>j:
            return False
    return True

#cdc必须开
def cdc_op(O):
    return O[6]==1 and O[11]==1



In [12]:
def cp(O,Z,Q_dm,Q_im,M_dm,M_im,P):
    cp = np.zeros((1,19))
    Q_c_dm = np.zeros((19,1))
    Q_c_im = np.zeros((19,1))
    Q_c_dm = M_dm @ Q_dm.T
    Q_c_im = M_im @ Q_im.T
    Q_c = Q_c_dm[1:] + Q_c_im[1:]
    Q_c = np.insert(Q_c, Q_c_dm[0],6)
    Q_c = np.insert(Q_c, Q_c_im[0],11)
    for i in range(0,20):
        fl=Z[i]
        cp[i]=fl+O[i]+Q_c[i]*P[i,fl]
    return np.sum(cp), Q_c
    

In [14]:
#目标优化式
##处置成本
def ccl_cp(O,Z,Q,M_s,P):
    cp=np.zeros((1,19))
    tmp=M_s*Q
    tmp1=tmp.sum(axis=1)
    tmp1=tmp1.reshape((1,19))
    for i in range(0,19):
        fl=Z[i]
        cp.T[i]=fl+O[i]+tmp1.T[i]*P[i][fl]
    return cp.sum(axis=0)
#运输成本
def ccl_ct(k,Q,M_s,M_d,L_s,L_d):
    #ct1=np.zeros((1,20))
    sum=0
    tmp1 = M_s * Q * L_s * 1.25
    Q_c = Q @ M_s.T
    Q_c = np.delete(Q_c,k,axis=1) 
    tmp2= M_d * Q_c * L_d.T * 0.6
    sum+=tmp1.sum()+tmp2.sum()
    return sum

#总成本
def cal_all(M1,M2,M3,M4,O,Z):
    c=0
    #进口
    c+=ccl_cp(O,Z,Q_im,M1,Pf)+ccl_ct(10,Q_im,M1,M2,L_dts_1,L_ctd[0])
    #国产
    c+=ccl_cp(O,Z,Q_dm,M3,Pf)+ccl_ct(6,Q_dm,M3,M4,L_dts_2,L_ctd[1])
    #c += cp(O,Z,Q_dm,Q_im,M1,M3,Pf)

    #c += ccl_ct(Q_im,M1,M2,L_dts_1,L_ctd[0]) + ccl_ct(Q_dm,M3,M4,L_dts_2,L_ctd[1])
    return c

In [15]:
def test():
    M1t=np.random.randint(0,2,size=(19,106))
    M2t=np.random.randint(0,2,size=(1,18))
    M3t=np.random.randint(0,2,size=(19,106))
    M4t=np.random.randint(0,2,size=(1,18))
    Ot=np.random.randint(0,2,size=(20,1))
    Zt=np.random.randint(0,2,size=(20,1))
    a=cal_all(M1t,M2t,M3t,M4t,Ot,Zt)
    print(a)

test()


[1.85788275e+08 1.85788247e+08 1.85788262e+08 1.85788248e+08
 1.85788251e+08 1.85788270e+08 1.85788248e+08 1.85788252e+08
 1.85788255e+08 1.85788272e+08 1.85788265e+08 1.85788267e+08
 1.85788270e+08 1.85788273e+08 1.85788260e+08 1.85788260e+08
 1.85788249e+08 1.85788251e+08 1.85788269e+08]


In [16]:
from gurobipy import *
m=Model("ass_mov")
m.addVars(M1,M2,M3,M4,O,Z,vtype=GRB.BINARY)
m.update()
m.setObjective(cal_all(M1,M2,M3,M4,O,Z),GRB.MINIMIZE)
m.addConstrs(time_lim(M1,M3,Q_im,Q_dm))
m.addConstrs(cap_lim(M1,M3))
m.addConstrs(cdc_op(O))
m.optimize()
print("M1:",M1,"M2",M2)

ModuleNotFoundError: No module named 'gurobipy'